# 1 INTRODUCTION

#### SCENARIO & BACKGROUND

Bronx is a borough in New York- the financial capital of USA. The diverse neighborhoods offer a competitive but lucrative market for dining out and entertainment. However the market is fiercely competitive and rentals are high. Businesses require high turnovers for viability.

#### PROBLEM

The challenge is to find an opportunity that suits the interest and tastes of a neighborhood. At the same time the neighborhood must offer an opportunity gap and not be saturated or have highly popular similar venues.

#### REQUIREMENT

An entrepreneur based out of Bronx, is looking to start a business in the area and wants a neighborhood analysis to setup a new restaurant or entertainment joint. Having an ability to start any food or entertainment business, the idea is to look for opportunities in neighborhood and meet them with a new business venture.

# 2 DATA SECTION

#### DATA REQUIRED 

To attend to project requirements, we require data of New York neighborhoods by borough. 
We also require data of popular venues visited through Foursquare API with frequency of visit for each venue.   

#### DATA SOURCES

The data for New York boroughs and neighborhoods including Bronx is obtained from following site:
 https://cocl.us/new_york_dataset 
This basic data acquired from the above links is used to access Foursquare API data about venues in each neighborhood. The most common venues are added to dataframe basis frequency received from API data

#### DATA LIMITATIONS

Our project assumes that Foursquare user data represents the whole of the neighborhoods. Due to popularity and wide usage of this API in NY, this is a reasonably safe assumption of being able to obtain a wide and representative data set.

# 3 METHODOLOGY

This section represents the main component of the report where the data is gathered, prepared for analysis. The tools described are used here and the Notebook cells indicates the execution of steps.

#### STRATEGY

Strategy involves using data mentioned in Section 2 above. Neighborhood data will be used in tandem with foursquare API venue popularity data to classify neighborhoods.  

#### EXECUTION & ANALYSIS 

We will consider what are the 10 most popular venues by neighborhood within the Bronx borough. We will sort most popular venues basis frequency of visits as recorded by Foursquare API user data. 
We will then classify the neighborhoods into clusters using K-Means clustering to group similar neighborhoods and separate dissimilar neighborhoods. We will then examine similar neighborhoods within a cluster to understand favorite venues by frequency within clusters. 
To be considered an opportunity gap, the neighborhood selected must not have the high frequency venues of its cluster in its top 5 most frequency venues. If the high frequency venues of cluster are already in top 5 of a neighborhood, the opportunity may be considered as saturated due to presence of ample or highly favored venues which will create an entry barrier.

### "CODE"

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    li

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)


In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [12]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [13]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.8466508, -73.8785937.


In [14]:
# create map of Manhattan using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

In [17]:
CLIENT_ID = '3PQ24LXWSTX2RBVFQTFUOLCW25Z3FUUJXJXLX3V3ZGDSKO2V' # Foursquare ID
CLIENT_SECRET = 'RUAO2V1CO5SQ5YLHHWDJU5ZURSRZFKDKAAEHXY3Q5BTFYESO' # Foursquare Secret
VERSION = '20180605' # Foursquare API version


#### USE FOURSQUARE API QUERIES FOR VENUE POPULARITY

In [18]:
bronx_data.loc[0, 'Neighborhood']

'Wakefield'

## CATEGORIES OF VENUES IN BRONX NEIGHBORHOODS

In [21]:
results = requests.get(url).json()

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Rite Aid,Pharmacy,40.896649,-73.844846
2,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568
3,Shell,Gas Station,40.894187,-73.845862
4,Dunkin',Donut Shop,40.890459,-73.849089


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [26]:
print(bronx_venues.shape)
bronx_venues.head()

(1214, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [27]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,28,28,28,28,28,28
Baychester,24,24,24,24,24,24
Bedford Park,36,36,36,36,36,36
Belmont,100,100,100,100,100,100
Bronxdale,11,11,11,11,11,11
Castle Hill,10,10,10,10,10,10
City Island,28,28,28,28,28,28
Claremont Village,16,16,16,16,16,16
Clason Point,9,9,9,9,9,9


In [28]:
print('There are {} uniques categories.'.format(len(bronx_venues['Venue Category'].unique())))

There are 170 uniques categories.


In [29]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nail Salon,Nightclub,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [30]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nail Salon,Nightclub,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Allerton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.071429,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.035714,0.000000,0.000000,0.071429,0.035714,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.035714,0.0000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.00,0.142857,0.00,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.035714,0.035714,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Baychester,0.041667,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

### Top 10 venues in each neighborhood

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Chinese Restaurant,Supermarket,Department Store,Gas Station,Martial Arts Dojo,Breakfast Spot,Donut Shop,Spa
1,Baychester,Donut Shop,Accessories Store,Bus Stop,Mattress Store,Sandwich Place,Shopping Mall,Fast Food Restaurant,Men's Store,Mexican Restaurant,Electronics Store
2,Bedford Park,Diner,Chinese Restaurant,Mexican Restaurant,Pizza Place,Deli / Bodega,Supermarket,Sandwich Place,Pharmacy,Burger Joint,Pub
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Grocery Store,Donut Shop,Dessert Shop,Bank,Shoe Store,Food & Drink Shop
4,Bronxdale,Breakfast Spot,Italian Restaurant,Mexican Restaurant,Spanish Restaurant,Eastern European Restaurant,Supermarket,Chinese Restaurant,Gym,Bank,Pizza Place


#### Cluster Neighborhoods

In [37]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 1, 4, 1, 2, 3, 2], dtype=int32)

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Ice Cream Shop,Laundromat,Dessert Shop,Donut Shop,Sandwich Place,Caribbean Restaurant,Gas Station,Pharmacy,Gym / Fitness Center,Gym
1,Bronx,Co-op City,40.874294,-73.829939,2,Baseball Field,Bus Station,Ice Cream Shop,Grocery Store,Mattress Store,Fast Food Restaurant,Discount Store,Park,Pharmacy,Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,2,Caribbean Restaurant,Deli / Bodega,Diner,Metro Station,Bowling Alley,Cosmetics Shop,Platform,Convenience Store,Donut Shop,Chinese Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,0,Plaza,Bus Station,River,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor
4,Bronx,Riverdale,40.890834,-73.912585,4,Bus Station,Park,Food Truck,Baseball Field,Home Service,Plaza,Gym,Bank,Fish & Chips Shop,Fast Food Restaurant


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# RESULTS

#### ANALYSIS OF EACH CLUSTER

In [44]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Bus Station,River,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor


Difficult to predict with just one neighborhood

In [46]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Woodlawn,Deli / Bodega,Pub,Pizza Place,Playground,Food & Drink Shop,Bar,Liquor Store,Italian Restaurant,Plaza,Moving Target
10,Pelham Parkway,Pizza Place,Frozen Yogurt Shop,Italian Restaurant,Ice Cream Shop,Bakery,Eye Doctor,Sandwich Place,Performing Arts Venue,Donut Shop,Chinese Restaurant
11,City Island,Harbor / Marina,Seafood Restaurant,Park,Thrift / Vintage Store,Ice Cream Shop,Deli / Bodega,Smoke Shop,Sandwich Place,Pizza Place,Diner
27,Throgs Neck,Deli / Bodega,Pizza Place,Asian Restaurant,Sports Bar,Bar,Juice Bar,Italian Restaurant,Coffee Shop,American Restaurant,Dive Bar
28,Country Club,Sandwich Place,Playground,Mobile Phone Shop,Italian Restaurant,Flea Market,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor
31,Van Nest,Pizza Place,Deli / Bodega,Bakery,Food Truck,Middle Eastern Restaurant,Donut Shop,Playground,Coffee Shop,Chinese Restaurant,Caribbean Restaurant
32,Morris Park,Pizza Place,Deli / Bodega,Burger Joint,Bakery,Juice Bar,Bagel Shop,Donut Shop,Supermarket,Buffet,Bar
33,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Grocery Store,Donut Shop,Dessert Shop,Bank,Shoe Store,Food & Drink Shop
35,North Riverdale,Pizza Place,Grocery Store,Bank,Italian Restaurant,Ice Cream Shop,Chinese Restaurant,Social Club,Burger Joint,Bus Station,Sandwich Place
38,Edgewater Park,Deli / Bodega,Italian Restaurant,Pizza Place,Coffee Shop,Chinese Restaurant,Juice Bar,Donut Shop,Pub,Park,Liquor Store


*Reasonable size of neighborhood clusters with similar popularity of venues is seen in cluster 2*

Pizza places occur frequently as a popular venue. Opportunity exists in locations like Bronxdale,Country Club & City Island neighborhoods which do not have pizza place in top 5.

Second most popular are Deli/Bodega with opportunity gaps in Pelham Parkway, City Island, Country Club, North Riverdale, Edenwald & Bronxdale

Third most popular are Italian restaurants with opportunities in 7 of the neighborhoods.

In [47]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Ice Cream Shop,Laundromat,Dessert Shop,Donut Shop,Sandwich Place,Caribbean Restaurant,Gas Station,Pharmacy,Gym / Fitness Center,Gym
1,Co-op City,Baseball Field,Bus Station,Ice Cream Shop,Grocery Store,Mattress Store,Fast Food Restaurant,Discount Store,Park,Pharmacy,Restaurant
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Metro Station,Bowling Alley,Cosmetics Shop,Platform,Convenience Store,Donut Shop,Chinese Restaurant
5,Kingsbridge,Pizza Place,Bar,Latin American Restaurant,Mexican Restaurant,Sandwich Place,Bakery,Supermarket,Fried Chicken Joint,Pharmacy,Discount Store
7,Norwood,Pizza Place,Park,Bank,Pharmacy,Chinese Restaurant,Pet Store,Coffee Shop,Plaza,Spanish Restaurant,Restaurant
8,Williamsbridge,Dance Studio,Caribbean Restaurant,Bar,Nightclub,Soup Place,Department Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
9,Baychester,Donut Shop,Accessories Store,Bus Stop,Mattress Store,Sandwich Place,Shopping Mall,Fast Food Restaurant,Men's Store,Mexican Restaurant,Electronics Store
12,Bedford Park,Diner,Chinese Restaurant,Mexican Restaurant,Pizza Place,Deli / Bodega,Supermarket,Sandwich Place,Pharmacy,Burger Joint,Pub
13,University Heights,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Donut Shop,Latin American Restaurant,Supermarket,Bank,Bakery,Shoe Store
15,Fordham,Mobile Phone Shop,Pizza Place,Donut Shop,Shoe Store,Bank,Gym / Fitness Center,Fast Food Restaurant,Pharmacy,Spanish Restaurant,Supermarket


* With 34 similar neighborhoods, we may look at sub-classifying cluster 3  further before exploring opportunities in these neighborhoods*

With present analysis the 7 most popular venues in order starting from most popular are Donut Shop, Pizza place, Supermarket, Pharmacy, Chinese Restaurant, Fast Food Restaurant and Sandwich place. We will ignore Supermarket & Pharmacy and consider the remaining 5 possibilities as our scope is limited to Food or entertainment business.

In [48]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Clason Point,Park,South American Restaurant,Pool,Boat or Ferry,Bus Stop,Grocery Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
34,Spuyten Duyvil,Park,Pharmacy,Grocery Store,Bus Line,Scenic Lookout,Bank,Thai Restaurant,Tennis Stadium,Distillery,Eastern European Restaurant


In [49]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Riverdale,Bus Station,Park,Food Truck,Baseball Field,Home Service,Plaza,Gym,Bank,Fish & Chips Shop,Fast Food Restaurant
14,Morris Heights,Spanish Restaurant,Cosmetics Shop,Pharmacy,Bus Station,Pizza Place,Park,Plaza,Grocery Store,Bank,Latin American Restaurant
39,Castle Hill,Pizza Place,Bank,Diner,Cosmetics Shop,Park,Bus Station,Deli / Bodega,Market,Baseball Field,Pharmacy


# DISCUSSIONS / RECOMMENDATIONS

Reasonable size of neighborhood clusters with similar popularity of venues is seen in cluster 2. This makes an ideal target geography to initiate business project. 
With 34 similar neighborhoods, we may look at sub-classifying cluster 3 further before exploring opportunities in these neighborhoods. Clusters 1, 4 and 5 have between 1-3 neighborhoods and hence we will not prioritize these options as they limit geography and also may not provide ample opportunity.


* Pizza places occur frequently as a popular venue. Opportunity exists in locations like Bronxdale,Country Club & City Island neighborhoods which do not have pizza place in top 5.

* Second most popular are Deli/Bodega with opportunity gaps in Pelham Parkway, City Island, Country Club, North Riverdale, Edenwald & Bronxdale

* Third most popular are Italian restaurants with opportunities in 7 of the neighborhoods.

* With present analysis of cluster 3, the 7 most popular venues in order starting from most popular are Donut Shop, Pizza place, Supermarket, Pharmacy, Chinese Restaurant, Fast Food Restaurant and Sandwich place. We will ignore Supermarket & Pharmacy and consider the remaining 5 possibilities as our scope is limited to Food or entertainment business.

# CONCLUSIONS AND FUTURE DIRECTIONS

* Initial analysis provides a very reasonable geography of 10 neighborhoods within cluster 2 to explore rental properties for business with 3 popular venue options. 
* Future sub-classification of cluster 3 can provide more options if required.
* Other data that can be explored include neighborhood demographics like age & preference for ethnic cuisine.